In [17]:
from langchain.vectorstores.cassandra import Cassandra 
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import Ollama

from datasets import load_dataset

# Used for DB connection
import cassio

from PyPDF2 import PdfReader

In [ ]:
ASTRA_DB_APPLICATION_TOKEN = ""
ASTRA_DB_ID = ""
HF_API_KEY = "" 

In [5]:
pdf_reader = PdfReader("apjspeech.pdf")

In [6]:
raw_text = ''
for i,page in enumerate(pdf_reader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

raw_text

'A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth of our country. During my intera ction at \nRashtrapati Bhavan in Delhi and at every state and union territor y as well as through my \nonline interactions, I have many unique experiences to share with you, which signify the \nfollowing important messages: \n \n1. Accelerate development : Aspiration of th

In [7]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

In [18]:
llm = Ollama(model="llama3.2:1b")

C:\Users\indra\AppData\Local\Temp\ipykernel_25928\1940434285.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2:1b")


In [11]:
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

astra_vector_store = Cassandra(
    embedding=embedding,
    table_name="qa_mini_demo",
    session=None,
    keyspace=None
)

C:\Users\indra\AppData\Local\Temp\ipykernel_25928\2047141792.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [13]:
from langchain.text_splitter import CharacterTextSplitter 

text_splitter = CharacterTextSplitter(
    separator="\n", chunk_size=800, chunk_overlap=200, length_function=len 
)
texts = text_splitter.split_text(raw_text)
texts

['A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth of our country. During my intera ction at',
 'home makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth of our country. During my intera ction at \nRashtrapati Bhavan in Delhi and at every state and union territor y as well as through my \nonline

In [14]:
texts[:50]

['A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth of our country. During my intera ction at',
 'home makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth of our country. During my intera ction at \nRashtrapati Bhavan in Delhi and at every state and union territor y as well as through my \nonline

In [16]:
# Load dataset into the vector store 
astra_vector_store.add_texts(texts[:50])
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)
astra_vector_index

VectorStoreIndexWrapper(vectorstore=<langchain_community.vectorstores.cassandra.Cassandra object at 0x0000026C2D0036D0>)

In [ ]:
query_text = "What does the speech mention about agriculture?"
answer = astra_vector_index.query(query_text, llm=llm)
answer

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


"The speech mentions nothing about technology. It discusses various goals and aspirations for India's development, including reducing divisions between rural and urban areas, improving access to energy and water, and promoting a harmonious economy through cooperation among agriculture, industry, and service sectors."

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


In [20]:
for doc,score in astra_vector_store.similarity_search_with_score(query_text, k=4):
    print(score, doc.page_content)

0.6874779277288254 institutions. Our country is blessed with natural resources, has shown c onsiderable 
progress in the last sixty years, and above all we have hard wor king people particularly 
the power of the 540 million youth of the country. Every sector of our count ry has given 
me the 
confidence that India can become a developed nation well before 2020. Whomsoeve r, I 
met they constantly ask what they can give to the nation. We should c onstantly strive to 
empower such members of the society. With this spirit, I am extr emely happy that we are 
on the right path. Here I am reminded of a famous poem: 
"When you wish upon a star, 
Makes no difference who you are, 
Anything your heart desires, 
Will come to you”
0.6874779277288254 institutions. Our country is blessed with natural resources, has shown c onsiderable 
progress in the last sixty years, and above all we have hard wor king people particularly 
the power of the 540 million youth of the country. Every sector of our cou